In [1]:
from dataset_prepare.smt_preprocess import get_assertions
import csv
from PIL import Image

In [2]:
import math
import numpy as np
import torchvision.transforms as transforms

In [3]:
def create_image(path):
    assertions = get_assertions(path)
    ascii_asserts = [ord(c) for c in assertions]
    n = math.ceil(math.sqrt(len(ascii_asserts)))
    padded_asserts = ascii_asserts + [0] * (n * n - len(ascii_asserts))
    image = np.array(padded_asserts).reshape(n, n)
    return image

In [4]:
train_image_dataset = []
with open('/Users/zhengyanglumacmini/Desktop/Projects/lm_smt/bin_sage2/raw/merged_3_mini_train.csv', 'r') as f:
    reader = csv.reader(f)
    # skip header
    next(reader)
    for row in reader:
        path = row[0]
        image = create_image(path)
        train_image_dataset.append(image)

In [5]:
z3_labels = []
cvc5_labels = []
with open('/Users/zhengyanglumacmini/Desktop/Projects/lm_smt/bin_sage2/raw/merged_3_mini_train.csv', 'r') as f:
    reader = csv.reader(f)
    # skip header
    next(reader)
    for row in reader:
        if row[1] == 'True':
            z3_labels.append(1)
        else:
            z3_labels.append(0)
        if row[2] == 'True':
            cvc5_labels.append(1)
        else:
            cvc5_labels.append(0)

In [6]:
trained_pil_images = []
for image in train_image_dataset:
    image_uint8 = np.uint8(image)
    pil_image = Image.fromarray(image_uint8)
    trained_pil_images.append(pil_image)


In [5]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [8]:
train_transformed_image_dataset = [transform(image) for image in trained_pil_images]

In [6]:
import torch
import torchvision.models as models
from torch import nn
from torch.optim import Adam


In [7]:
import torch
from torch.utils.data import Dataset, DataLoader

In [8]:
class CustomDataset(Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        image = self.images[index].repeat(3, 1, 1)
        label = self.labels[index]
        return image, label

In [12]:
z3_train_dataset = CustomDataset(train_transformed_image_dataset, torch.tensor(z3_labels, dtype=torch.float32))
z3_train_dataloader = DataLoader(z3_train_dataset, batch_size=32, shuffle=True)

In [13]:
eff_model = models.efficientnet_b0(pretrained=False)
num_ftrs = eff_model.classifier[1].in_features
eff_model.classifier[1] = nn.Linear(num_ftrs, 1)

eff_model = nn.Sequential(eff_model, nn.Sigmoid())

/opt/anaconda3/envs/machsmt/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/envs/machsmt/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [14]:
criterion = nn.BCELoss()
optimizer = Adam(eff_model.parameters(), lr=0.001)

In [15]:
num_epochs = 3
eff_model.train()
for epoch in range(num_epochs):
    for images, labels in z3_train_dataloader:
        optimizer.zero_grad()
        outputs = eff_model(images)
        loss = criterion(outputs, labels.unsqueeze(1))
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch} loss: {loss.item()}')

Epoch 0 loss: 0.3874310851097107
Epoch 1 loss: 0.5023546814918518
Epoch 2 loss: 0.5212305784225464


In [17]:
# save model
torch.save(eff_model.state_dict(), './eff_model_z3.pt')

In [4]:
test_image_dataset = []
z3_labels_test = []
with open('/Users/zhengyanglumacmini/Desktop/Projects/lm_smt/bin_sage2/raw/merged_3_mini_test.csv', 'r') as f:
    reader = csv.reader(f)
    # skip header
    next(reader)
    for row in reader:
        path = row[0]
        print(path)
        image = create_image(path)
        test_image_dataset.append(image)
        if row[1] == 'True':
            z3_labels_test.append(1)
        else:
            z3_labels_test.append(0)


/Users/zhengyanglumacmini/Desktop/smt-lib/QF_BV/Sage2/bench_7148.smt2
/Users/zhengyanglumacmini/Desktop/smt-lib/QF_BV/Sage2/bench_8248.smt2
/Users/zhengyanglumacmini/Desktop/smt-lib/QF_BV/Sage2/bench_10233.smt2
/Users/zhengyanglumacmini/Desktop/smt-lib/QF_BV/Sage2/bench_3162.smt2
/Users/zhengyanglumacmini/Desktop/smt-lib/QF_BV/Sage2/bench_1056.smt2
/Users/zhengyanglumacmini/Desktop/smt-lib/QF_BV/Sage2/bench_16996.smt2
/Users/zhengyanglumacmini/Desktop/smt-lib/QF_BV/Sage2/bench_2915.smt2
/Users/zhengyanglumacmini/Desktop/smt-lib/QF_BV/Sage2/bench_1590.smt2
/Users/zhengyanglumacmini/Desktop/smt-lib/QF_BV/Sage2/bench_9516.smt2
/Users/zhengyanglumacmini/Desktop/smt-lib/QF_BV/Sage2/bench_16438.smt2
/Users/zhengyanglumacmini/Desktop/smt-lib/QF_BV/Sage2/bench_13569.smt2
/Users/zhengyanglumacmini/Desktop/smt-lib/QF_BV/Sage2/bench_2326.smt2
/Users/zhengyanglumacmini/Desktop/smt-lib/QF_BV/Sage2/bench_5701.smt2
/Users/zhengyanglumacmini/Desktop/smt-lib/QF_BV/Sage2/bench_3022.smt2
/Users/zhengyang

In [10]:
test_image_dataset = [Image.fromarray(np.uint8(image)) for image in test_image_dataset]
test_image_dataset = [transform(image) for image in test_image_dataset]

In [11]:
z3_test_dataset = CustomDataset(test_image_dataset, torch.tensor(z3_labels_test, dtype=torch.float32))
z3_test_dataloader = DataLoader(z3_test_dataset, batch_size=8, shuffle=False)

In [12]:
# load model from './eff_model_z3.pt'
eff_model = models.efficientnet_b0(pretrained=False)
num_ftrs = eff_model.classifier[1].in_features
eff_model.classifier[1] = nn.Linear(num_ftrs, 1)
eff_model = nn.Sequential(eff_model, nn.Sigmoid())
eff_model.load_state_dict(torch.load('./eff_model_z3.pt'))


/opt/anaconda3/envs/machsmt/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/envs/machsmt/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


<All keys matched successfully>

In [14]:
# creat a list of predictions based on the trained model
predictions = []
eff_model.eval()
for images, labels in z3_test_dataloader:
    outputs = eff_model(images)
    # preds as a list of 0s and 1s
    preds = [1 if output > 0.5 else 0 for output in outputs]
    predictions.extend(preds)
    


In [15]:
predictions

[1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,


In [17]:
# get labels from the test dataset
labels = []
for _, label in z3_test_dataset:
    labels.append(int(label.item()))
print(labels)

[1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 

In [18]:
with open('z3_effnet_preds.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(["preds", "labels"])
    for pred, label in zip(predictions, labels):
        writer.writerow([pred, label])